---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [160]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.shape

(500,)

In [161]:
def date_sorter():
    
    pattern1 = '(?P<Date>(?P<Month>\d{1,2})[/-](?P<Day>\d{1,2})[/-](?P<Year>\d{2,4}))'
    df1 = df.str.extractall(pattern1)
    
    pattern2 = '(?P<Date>(?P<Month>Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December)[-|,|\s|.](?P<Day>\d{1,2}| \d{1,2})[-|,|\s](?P<Year>\d{2,4}| \d{2,4}))'
    df2 = df.str.extractall(pattern2)
    
    pattern3 = '(?P<Date>(?P<Day>\d{1,2})[\s|,|-|.](?P<Month>Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December)[\s|,|-|.](?P<Year>\d{2,4}| \d{2,4}))'
    df3 = df.str.extractall(pattern3)
    df3 = df3[['Date','Month','Day','Year']]
    
    pattern4 = '(?P<Date>(?P<Month>Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December)[\s|,|.|-](?P<Day>\w{4}| \w{4})[\s|,|.|-](?P<Year>\d{2,4}| \d{2,4}))'
    df4 = df.str.extractall(pattern4)
    
    pattern5 = '(?P<Date>(?P<Month>Jan|January|Janaury|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December|Decemeber)[\s|,|.|-](?P<Year>\d{2,4}| \d{2,4}))'
    df5 = df.iloc[228:343].str.extractall(pattern5)
    df5['Day'] = 1
    df5 = df5[['Date','Month','Day','Year']]
    
    pattern6 = '(?P<Date>(?P<Month>1|2|3|4|5|6|7|8|9|10|11|12)[/](?P<Year>\d{4}))'
    df6 = df.iloc[343:455].str.extractall(pattern6)
    df6['Day'] = 1
    df6 = df6[['Date','Month','Day','Year']]
    
    pattern7 = '(?P<Date>(?P<Year>\d{4}))'
    df7 = df.iloc[455:].str.extractall(pattern7)
    df7['Month'] = 1
    df7['Day'] = 1
    df7 = df7[['Date','Month','Day','Year']]
    
    df_new = df1.append(df3)
    df_new = df_new.append(df2)
    df_new = df_new.append(df5)
    df_new = df_new.append(df6)
    df_new = df_new.append(df7)
    
    lst_month = []

    for i in df_new['Month']:
        if i=='Jan' or i=='January' or i=='Janaury' or i=='1' or i==1 or i=='01':
            lst_month.append(1)
        elif i=='Feb' or i=='February' or i==2 or i=='2' or i=='02':
            lst_month.append(2)
        elif i=='Mar' or i=='March' or i==3 or i=='3' or i=='03':
            lst_month.append(3)
        elif i=='Apr' or i=='April' or i==4 or i=='4' or i=='04':
            lst_month.append(4)
        elif i=='May' or i==5 or i=='5' or i=='05':
            lst_month.append(5)
        elif i=='Jun' or i=='June' or i==6 or i=='6' or i=='06':
            lst_month.append(6)
        elif i=='Jul' or i=='July' or i==7 or i=='7' or i=='07':
            lst_month.append(7)
        elif i=='Aug' or i=='August' or i==8 or i=='8' or i=='08':
            lst_month.append(8)
        elif i=='Sep' or i=='September' or i==9 or i=='9' or i=='09':
            lst_month.append(9)
        elif i=='Oct' or i=='October' or i==10 or i=='10' or i=='10':
            lst_month.append(10)
        elif i=='Nov' or i=='November' or i==11 or i=='11' or i=='11':
            lst_month.append(11)
        elif i=='Dec' or i=='December' or i=='Decemeber' or i==12 or i=='12' or i=='12':
            lst_month.append(12)
    
    df_new['Month'] = lst_month
    df_new['Month'] = df_new['Month'].astype('int')
    df_new['Day'] = df_new['Day'].astype('int')
    df_new['Year'] = df_new['Year'].astype('int')
    df_new.drop('Date',axis=1,inplace=True)
    df_new = df_new.reset_index()
    
    df_new.rename(columns = {"Month":"month",
                        "Day":"day",
                        "Year":"year"},inplace=True)
    
    year = []
    for i in df_new.year:
        if i in range(0,21):
            year.append(int('20'+str(i)))
        elif i in range(21,100):
            year.append(int('19'+str(i)))
        else:
            year.append(i)
    
    df_new['year'] = year
    df_new = df_new.drop(73).reset_index()
    
    df_new['Date'] = pd.to_datetime(df_new[['year','month','day']])
    
    df_new.sort_values(by=['Date'],inplace=True)
    df_new.reset_index(drop=True)
    df_output = pd.Series(df_new.index)
    
    
    return df_output

date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [3]:
text1 = pd.Series('04/20/2009; 04/20/09; 4/20/09; 4/3/09')

pattern1 = '(?P<Date>(?P<Month>\d{1,2})[/-](?P<Day>\d{1,2})[/-](?P<Year>\d{2,4}))'
text1.str.extractall(pattern1)

Date Month Day  Year
  match                            
0 0      04/20/2009    04  20  2009
  1        04/20/09    04  20    09
  2         4/20/09     4  20    09
  3          4/3/09     4   3    09

In [4]:
df1 = df.str.extractall(pattern1)
df1.shape

(126, 4)

In [5]:
text2 = pd.Series('Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009')

pattern2 = '(?P<Date>(?P<Month>Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December)[-|,|\s|.](?P<Day>\d{1,2}| \d{1,2})[-|,|\s](?P<Year>\d{2,4}| \d{2,4}))'
text2.str.extractall(pattern2)

Date  Month  Day   Year
  match                                   
0 0         Mar-20-2009    Mar   20   2009
  1        Mar 20, 2009    Mar   20   2009
  2      March 20, 2009  March   20   2009
  3       Mar. 20, 2009    Mar   20   2009
  4         Mar 20 2009    Mar   20   2009

In [6]:
df2 = df.str.extractall(pattern2)
df2.shape

(34, 4)

In [7]:
text3 = pd.Series('20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009')

pattern3 = '(?P<Date>(?P<Day>\d{1,2})[\s|,|-|.](?P<Month>Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December)[\s|,|-|.](?P<Year>\d{2,4}| \d{2,4}))'

text3.str.extractall(pattern3)


Date Day  Month   Year
  match                                  
0 0         20 Mar 2009  20    Mar   2009
  1       20 March 2009  20  March   2009
  2        20 Mar. 2009  20    Mar   2009
  3      20 March, 2009  20  March   2009

In [8]:
df3 = df.str.extractall(pattern3)
df3 = df3[['Date','Month','Day','Year']]
df3.shape

(69, 4)

In [9]:
text4 = pd.Series('Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009')

pattern4 = '(?P<Date>(?P<Month>Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December)[\s|,|.|-](?P<Day>\w{4}| \w{4})[\s|,|.|-](?P<Year>\d{2,4}| \d{2,4}))'

text4.str.extractall(pattern4)

Date Month   Day   Year
  match                                   
0 0      Mar 20th, 2009   Mar  20th   2009
  1      Mar 21st, 2009   Mar  21st   2009
  2      Mar 22nd, 2009   Mar  22nd   2009

In [10]:
df4 = df.str.extractall(pattern4)
df4

,Date,Month,Day,Year


In [74]:
text5 = pd.Series('Feb 2009; Sep 2009; Oct 2010')

pattern5 = '(?P<Date>(?P<Month>Jan|January|Janaury|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December|Decemeber)[\s|,|.|-](?P<Year>\d{2,4}| \d{2,4}))'

text5.str.extractall(pattern5)

Date Month  Year
  match                      
0 0      Feb 2009   Feb  2009
  1      Sep 2009   Sep  2009
  2      Oct 2010   Oct  2010

In [75]:
df5 = df.iloc[228:343].str.extractall(pattern5)
df5['Day'] = 1
df5 = df5[['Date','Month','Day','Year']]
df5.shape

(115, 4)

Date     January 2007
Month         January
Day                 1
Year             2007
Name: (314, 0), dtype: object

In [13]:
text6 = pd.Series('6/2008; 12/2009')

pattern6 = '(?P<Date>(?P<Month>1|2|3|4|5|6|7|8|9|10|11|12)[/](?P<Year>\d{4}))'

text6.str.extractall(pattern6)

Date Month  Year
  match                     
0 0       6/2008     6  2008
  1      12/2009    12  2009

In [14]:
df6 = df.iloc[343:455].str.extractall(pattern6)
df6['Day'] = 1
df6 = df6[['Date','Month','Day','Year']]
df6.shape

(112, 4)

In [15]:
text7 = pd.Series('2009; 2010')

pattern7 = '(?P<Date>(?P<Year>\d{4}))'

text7.str.extractall(pattern7)

Date  Year
  match            
0 0      2009  2009
  1      2010  2010

In [16]:
df7 = df.iloc[455:].str.extractall(pattern7)
df7['Month'] = 1
df7['Day'] = 1
df7 = df7[['Date','Month','Day','Year']]
df7.shape

(45, 4)

In [130]:
df_new = df1.append(df3)
df_new = df_new.append(df2)
df_new = df_new.append(df5)
df_new = df_new.append(df6)
df_new = df_new.append(df7)
df_new.shape

(501, 4)

In [131]:
df_new.head()

,,Date,Month,Day,Year
,match,,,,
0,0,03/25/93,03,25,93
1,0,6/18/85,6,18,85
2,0,7/8/71,7,8,71
3,0,9/27/75,9,27,75
4,0,2/6/96,2,6,96


In [132]:
lst_month = []

for i in df_new['Month']:
    if i=='Jan' or i=='January' or i=='Janaury' or i=='1' or i==1 or i=='01':
        lst_month.append(1)
    elif i=='Feb' or i=='February' or i==2 or i=='2' or i=='02':
        lst_month.append(2)
    elif i=='Mar' or i=='March' or i==3 or i=='3' or i=='03':
        lst_month.append(3)
    elif i=='Apr' or i=='April' or i==4 or i=='4' or i=='04':
        lst_month.append(4)
    elif i=='May' or i==5 or i=='5' or i=='05':
        lst_month.append(5)
    elif i=='Jun' or i=='June' or i==6 or i=='6' or i=='06':
        lst_month.append(6)
    elif i=='Jul' or i=='July' or i==7 or i=='7' or i=='07':
        lst_month.append(7)
    elif i=='Aug' or i=='August' or i==8 or i=='8' or i=='08':
        lst_month.append(8)
    elif i=='Sep' or i=='September' or i==9 or i=='9' or i=='09':
        lst_month.append(9)
    elif i=='Oct' or i=='October' or i==10 or i=='10' or i=='10':
        lst_month.append(10)
    elif i=='Nov' or i=='November' or i==11 or i=='11' or i=='11':
        lst_month.append(11)
    elif i=='Dec' or i=='December' or i=='Decemeber' or i==12 or i=='12' or i=='12':
        lst_month.append(12)
        
len(lst_month)

501

In [133]:
df_new['Month'] = lst_month
df_new['Month'] = df_new['Month'].astype('int')
df_new['Day'] = df_new['Day'].astype('int')
df_new['Year'] = df_new['Year'].astype('int')
df_new.drop('Date',axis=1,inplace=True)
df_new.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 501 entries, (0, 0) to (499, 0)
Data columns (total 3 columns):
Month    501 non-null int64
Day      501 non-null int64
Year     501 non-null int64
dtypes: int64(3)
memory usage: 17.2 KB


In [134]:
df_new = df_new.reset_index()

In [135]:
#df_new = df_new[['Month','Day','Year']]
df_new.rename(columns = {"Month":"month",
                        "Day":"day",
                        "Year":"year"},inplace=True)
df_new.head()

,level_0,match,month,day,year
0,0,0,3,25,93
1,1,0,6,18,85
2,2,0,7,8,71
3,3,0,9,27,75
4,4,0,2,6,96


In [136]:
year = []
for i in df_new.year:
    if i in range(0,21):
        year.append(int('20'+str(i)))
    elif i in range(21,100):
        year.append(int('19'+str(i)))
    else:
        year.append(i)
len(year)        

501

In [137]:
df_new['year'] = year
df_new.head()

,level_0,match,month,day,year
0,0,0,3,25,1993
1,1,0,6,18,1985
2,2,0,7,8,1971
3,3,0,9,27,1975
4,4,0,2,6,1996


In [138]:
df_new['year'] = year
df_new = df_new.drop(73).reset_index()
df_new.shape

(500, 6)

In [144]:
df_new['Date'] = pd.to_datetime(df_new[['year','month','day']])
df_new.head()

,index,level_0,match,month,day,year,Date
9,9,9,0,4,10,1971,1971-04-10
84,85,84,0,5,18,1971,1971-05-18
2,2,2,0,7,8,1971,1971-07-08
53,53,53,0,7,11,1971,1971-07-11
28,28,28,0,9,12,1971,1971-09-12


In [147]:
df_new.sort_values(by=['Date'],inplace=True)

In [148]:
df_new.sort_values(by=['Date'],inplace=True)
df_new.reset_index(drop=True)

,index,level_0,match,month,day,year,Date
0,9,9,0,4,10,1971,1971-04-10
1,85,84,0,5,18,1971,1971-05-18
2,2,2,0,7,8,1971,1971-07-08
3,53,53,0,7,11,1971,1971-07-11
4,28,28,0,9,12,1971,1971-09-12
5,475,474,0,1,1,1972,1972-01-01
6,154,153,0,1,13,1972,1972-01-13
7,13,13,0,1,26,1972,1972-01-26
8,130,129,0,5,6,1972,1972-05-06
9,99,98,0,5,13,1972,1972-05-13


In [158]:
df_new.sort_values(by=['Date'],inplace=True)
df_new.reset_index(drop=True)
df_output = pd.Series(df_new.index)

In [159]:
df_output

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [106]:
def date_sorter():
    
    import numpy as np
    import re

    # Your code here
#     Testing Data
#     df = ["•04/20/2009;", "04/20/09;", "4/20/09;", "4/3/09;",
#     "•Mar-20-2009;", "Mar 20, 2009;", "March 20, 2009;", "Mar. 20, 2009;", "Mar 20 2009;", "October 14 1974",
#     "•20 Mar 2009;", "20 March 2009;", "20 Mar. 2009;", "20 March, 2009","2June, 1999",
#     "•Mar 20th, 2009;", "Mar 21st, 2009;", "Mar 22nd, 2009",
#     "•Feb 2009;", "Sep 2009;", "Oct 2010",
#     "•6/2008;", "12/2009",
#     "•2009;", "2010"]
        
    # df = ["•04/20/2009;", "04/20/09;", "4/20/09;", "4/3/09;",
    pattern1 = r'(0?[1-9]|1[0-2])[\/\-](0?[1-9]|[12]\d|30|31)[\/\-](\d{4}|\d{2})'
    df1 = df.str.extractall(pattern1)
    df1.columns = ["month", "day", "year"]
    df1 = df1.reset_index()
    #df1
    
    #"•Mar-20-2009;", "Mar 20, 2009;", "March 20, 2009;", "Mar. 20, 2009;", "Mar 20 2009;",
    #October 14 1974
    #"•Mar 20th, 2009;", "Mar 21st, 2009;", "Mar 22nd, 2009",
    #pattern2 = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.]*[ -](\d{1,2})[a-z]{0,2},[ -](\d{4})'
    pattern2 = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.]*[ -](\d{1,2})[a-z\.\,]*[ -](\d{4})'
    df2=df.str.extractall(pattern2)
    df2.columns = ["month", "day", "year"]
    df2 = df2.reset_index()
    #df2
    
    #"•20 Mar 2009;", "20 March 2009;", "20 Mar. 2009;", "20 March, 2009","2June, 1999",
    # "•Feb 2009;", "Sep 2009;", "Oct 2010",
    pattern3 = r'(\d{1,2})?[ -]?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.\,]*[ -](\d{4})'
    df3=df.str.extractall(pattern3)
    df3.columns = ["day", "month", "year"]
    df3 = df3.reset_index()
    #df3
    
    
    # "•6/2008;", "12/2009",
    pattern4 = r'(\d{1,2})[/](\d{4})'
    df4 = df.str.extractall(pattern4)
    df4.insert(0, column='day', value=np.nan)
    df4.columns = ["day" , "month", "year"]
    df4 = df4.reset_index()
    #df4
    
    ## "•2009;", "2010"
    pattern5 = r'(\d{4})'
    df5 = df.str.extractall(pattern5)
    df5.insert(0, column='day', value=np.nan)
    df5.insert(1, column='month', value=np.nan)
    df5.columns = ["month", "day", "year"]
    df5 = df5=df5.reset_index()
    #df5
    
    output = df1.append(df2[~df2.level_0.isin(df1.level_0)])
    #output.shape
    output = output.append(df3[~df3.level_0.isin(output.level_0)])
    #output.shape
    output = output.append(df4[~df4.level_0.isin(output.level_0)])
    #output.shape
    output = output.append(df5[~df5.level_0.isin(output.level_0)])
    #output.shape
    
    output = pd.DataFrame(output,columns = ["level_0", "match", "day", "month","year"])
    output.year = np.where(output.year.apply(len)==2, "19"+output.year, output.year)
    output = output.fillna("1")
    
    month_replace ={
            'Jan' : 1,
            'Feb' : 2,
            'Mar' : 3,
            'Apr' : 4,
            'May' : 5,
            'Jun' : 6,
            'Jul' : 7,
            'Aug' : 8,
            'Sep' : 9, 
            'Oct' : 10,
            'Nov' : 11,
            'Dec' : 12
    }
    
    output.month = output.month.replace(month_replace)
    
    output.day = output.day.astype(int)
    output.month = output.month.astype(int)
    output.year = output.year.astype(int)
    
    output["date"] = pd.to_datetime(output.loc[:,["year", "month", "day"]])
    
    output = output.sort_values(["date", "level_0"]).reset_index(drop=True)
    #output.info()
    #return_value = pd.Series(output.level_0, name="index")
    
    return output#return_value # Your answer here

In [107]:
date_sorter()

,level_0,match,day,month,year,date
0,9,0,10,4,1971,1971-04-10
1,84,0,18,5,1971,1971-05-18
2,2,0,8,7,1971,1971-07-08
3,53,0,11,7,1971,1971-07-11
4,28,0,12,9,1971,1971-09-12
5,474,0,1,1,1972,1972-01-01
6,153,0,13,1,1972,1972-01-13
7,13,0,26,1,1972,1972-01-26
8,129,0,6,5,1972,1972-05-06
9,98,0,13,5,1972,1972-05-13
